In [1]:
# exploration of code with RAG

In [3]:
# from git import Repo


In [25]:
# Set env var OPENAI_API_KEY or load from a .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")
WEAVIATE_URL = os.getenv("WEAVIATE_URL")

In [9]:
# Clone
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path="/home/cmagganas/hackathon-nov23/AIM-Legendary-Hackathon-RAG/langchain")

In [11]:
# Load
GenericLoader.from_filesystem(
    "../langchain" + "/libs/langchain/langchain",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=250),
)
len(loader.load())

2512

In [12]:
documents[0]

Document(page_content='"""Keep here for backwards compatibility."""\nfrom langchain.chains.example_generator import generate_example\n\n__all__ = ["generate_example"]\n', metadata={'source': '../langchain/libs/langchain/langchain/example_generator.py', 'language': <Language.PYTHON: 'python'>})

In [13]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
len(python_splitter.split_documents(documents))

5785

In [14]:
texts[5]

Document(page_content='def _dump_generations_to_json(generations: RETURN_VAL_TYPE) -> str:\n    """Dump generations to json.\n\n    Args:\n        generations (RETURN_VAL_TYPE): A list of language model generations.\n\n    Returns:\n        str: Json representing a list of generations.\n\n    Warning: would not work well with arbitrary subclasses of `Generation`\n    """\n    return json.dumps([generation.dict() for generation in generations])', metadata={'source': '../langchain/libs/langchain/langchain/cache.py', 'content_type': 'functions_classes', 'language': <Language.PYTHON: 'python'>})

In [27]:
import weaviate

weaviate.Client(
  url=WEAVIATE_URL,
  auth_client_secret=auth_config,
  additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY  # Replace with your inference API key
weaviate.Client(
  url=WEAVIATE_URL,
  auth_client_secret=weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY),
  additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY  # Replace with your inference API key
    }
)

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Weaviate



embeddings = OpenAIEmbeddings()


db = Weaviate.from_documents(
    texts,
    embeddings, 
    client=client, 
    by_text=False
)
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-FJvIkHhCiV8P54k6tjSGdWgE on tokens per min (TPM): Limit 1000000, Used 815548, Requested 186127. Please try again in 100ms. Visit https://platform.openai.com/account/rate-limits to learn more..
